In [ ]:
#Instalacion de librerias
!pip install pydrive2

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Autenticacion y creacion de usuario (clientes) de PyDrive2
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file = drive.CreateFile({'id':''})
file.GetContentFile('____.zip')
!unzip ____.zip

In [ ]:
import os
import shutil
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model

def extract_features(folder_path, model, input_shape=(224, 224)):
    """Extracción de Características Usando el Modelo Pre-Entrenado."""
    features = []
    file_paths = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
            file_path = os.path.join(folder_path, filename)
            try:
                img = image.load_img(file_path, target_size=input_shape)
                img_data = image.img_to_array(img)
                img_data = np.expand_dims(img_data, axis=0)
                img_data = preprocess_input(img_data)

                #Extracción de Características
                feature = model.predict(img_data)
                features.append(feature.flatten())
                file_paths.append(file_path)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

    return np.array(features), file_paths

def cluster_images(features, n_clusters=5):

    """Agrupar la imagenes usando K-Means"""
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    labels = kmeans.fit_predict(features)
    return labels

def save_clustered_images(labels, file_paths, output_folder):

    """Guardar las imagenes en directorios diferentes basados en clustering."""
    os.makedirs(output_folder, exist_ok=True)

    for label in set(labels):
        cluster_folder = os.path.join(output_folder, f"cluster_{label}")
        os.makedirs(cluster_folder, exist_ok=True)

        for i, file_path in enumerate(file_paths):
            if labels[i] == label:
                shutil.copy(file_path, cluster_folder)
                print(f"Image {file_path} saved to {cluster_folder}")

def find_and_cluster_images(input_folder, output_folder, n_clusters=5):
    """Main function to extract features, cluster, and save images."""

    # Modelo Pre-Entrenado ResNet50
    base_model = ResNet50(weights='imagenet')
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

    #Extracción de características
    features, file_paths = extract_features(input_folder, model)

    if len(file_paths) == 0:
        print("No images found in the folder.")
        return

    labels = cluster_images(features, n_clusters=n_clusters)

    save_clustered_images(labels, file_paths, output_folder)


input_folder = '/content/data/'
output_folder = '/content/Zips'

find_and_cluster_images(input_folder, output_folder, n_clusters=7)